In [31]:
import numpy as np
import os

In [32]:
mode = "train"
eval_folder = f"/checkpoints/sticker/SFT/eval-{mode}"
res_folder = []
for subf in os.listdir(eval_folder):
    if subf.startswith("checkpoint"):
        res_folder.append(subf)

In [33]:
score_name = "select_scores_dm_scale7.0.npy"

In [34]:
all_select_scores = {}
for folder in res_folder:
    path = os.path.join(eval_folder, folder, score_name)
    select_scores = np.load(path, allow_pickle=True).item()
    all_select_scores[folder] = select_scores

In [35]:
example_key = list(sorted(all_select_scores.keys(), key=lambda x:float(x.split("-")[-1][4:])))[0]

In [37]:
example_key

'checkpoint-375-scale1.0-mask0.0'

In [39]:
re_select_scores = {}
for uid in example_scores:
    if uid not in re_select_scores:
        re_select_scores[uid] = {}

    for iid in example_scores[uid]:
        if iid not in re_select_scores[uid]:
            re_select_scores[uid][iid] = {}

        for gid in example_scores[uid][iid]:
            if gid not in re_select_scores[uid][iid]:
                re_select_scores[uid][iid][gid] = {}
                re_select_scores[uid][iid][gid]["folder"] = []
                re_select_scores[uid][iid][gid]["seq_id"] = []
                re_select_scores[uid][iid][gid]["img_id"] = []
                re_select_scores[uid][iid][gid]["score"] = []
            
            for folder in all_select_scores:
                cur_scores = all_select_scores[folder][uid][iid][gid]
                all_seq_id = []
                all_img_id = []
                all_seq_score = []
                for seq_id in cur_scores:
                    all_seq_id.append(seq_id)
                    seq_scores = cur_scores[seq_id]["hist_score"]
                    max_score = max(seq_scores)
                    max_img_idx = seq_scores.index(max_score)
                    all_img_id.append(max_img_idx)
                    all_seq_score.append(max_score)
                
                max_score = max(all_seq_score)
                max_idx = all_seq_score.index(max_score)
                max_seq_id = all_seq_id[max_idx]
                max_img_id = all_img_id[max_idx]

                re_select_scores[uid][iid][gid]["folder"].append(folder)
                re_select_scores[uid][iid][gid]["seq_id"].append(max_seq_id)
                re_select_scores[uid][iid][gid]["img_id"].append(max_img_id)
                re_select_scores[uid][iid][gid]["score"].append(max_score)

In [40]:
from copy import deepcopy

In [41]:
chosen_scores = deepcopy(re_select_scores)
rejected_scores = deepcopy(re_select_scores)

for uid in re_select_scores:
    for iid in re_select_scores[uid]:
        for gid in re_select_scores[uid][iid]:
            scores = re_select_scores[uid][iid][gid]["score"]
            best_score = max(scores)
            best_idx = scores.index(best_score)
            best_folder = re_select_scores[uid][iid][gid]["folder"][best_idx]
            best_seq_id = re_select_scores[uid][iid][gid]["seq_id"][best_idx]
            best_img_id = re_select_scores[uid][iid][gid]["img_id"][best_idx]

            chosen_scores[uid][iid][gid]["folder"] = best_folder
            chosen_scores[uid][iid][gid]["seq_id"] = best_seq_id
            chosen_scores[uid][iid][gid]["img_id"] = best_img_id
            chosen_scores[uid][iid][gid]["score"] = best_score

            worst_score = min(scores)
            worst_idx = scores.index(worst_score)
            rejected_scores[uid][iid][gid]["folder"] = re_select_scores[uid][iid][gid]["folder"][worst_idx]
            rejected_scores[uid][iid][gid]["seq_id"] = re_select_scores[uid][iid][gid]["seq_id"][worst_idx]
            rejected_scores[uid][iid][gid]["img_id"] = re_select_scores[uid][iid][gid]["img_id"][worst_idx]
            rejected_scores[uid][iid][gid]["score"] = worst_score

In [42]:
np.save(os.path.join(eval_folder, "chosen_scores.npy"), np.array(chosen_scores))
np.save(os.path.join(eval_folder, "rejected_scores.npy"), np.array(rejected_scores))

In [43]:
all_res = {}
res_name = "all_res_dm_scale7.0.npy"
for folder in res_folder:
    path = os.path.join(eval_folder, folder, res_name)
    res = np.load(path, allow_pickle=True).item()
    all_res[folder] = res

In [ ]:
import torch

data_seq = {}
preference_data = {}
seq_num = 0
for uid in chosen_scores:
    if uid not in data_seq:
        data_seq[uid] = {}
        preference_data[uid] = {}

    for iid in chosen_scores[uid]:
        if iid not in preference_data[uid]:
            preference_data[uid][iid] = {}

        for gid in chosen_scores[uid][iid]:
            if gid not in preference_data[uid][iid]:
                preference_data[uid][iid][gid] = {}
            
            if gid not in data_seq[uid]:
                data_seq[uid][gid] = []

            chosen_folder = chosen_scores[uid][iid][gid]["folder"]
            chosen_seq_id = chosen_scores[uid][iid][gid]["seq_id"]

            history = all_res[chosen_folder][uid][iid][gid]["history"].tolist()
            chosen = all_res[chosen_folder][uid][iid][gid]["seqs"][chosen_seq_id]["tokens"]
            non_pad_idx = torch.nonzero(chosen != 2, as_tuple=False)[-1].item()
            chosen = chosen[:non_pad_idx + 1]

            rejected_folder = rejected_scores[uid][iid][gid]["folder"]
            rejected_seq_id = rejected_scores[uid][iid][gid]["seq_id"]
            rejected = all_res[rejected_folder][uid][iid][gid]["seqs"][rejected_seq_id]["tokens"]
            non_pad_idx = torch.nonzero(rejected != 2, as_tuple=False)[-1].item()
            rejected = rejected[:non_pad_idx + 1]
            
            seq = history + [iid]
            data_seq[uid][gid].append(seq)
            preference_data[uid][iid][gid]["chosen"] = chosen
            preference_data[uid][iid][gid]["rejected"] = rejected

            seq_num += 1

In [45]:
np.save(f"{data_path}/dpo/{mode}.npy", np.array(data_seq))
np.save(f"{data_path}/dpo/{mode}_preference_data.npy", np.array(preference_data))